In [2]:
import minio
from minio.error import S3Error
from minio.commonconfig import ENABLED
from minio.versioningconfig import VersioningConfig

import pathlib
import pandas as pd
import os
import json
from tqdm import tqdm

from minio_utils import * 
#####-------------------------------------------------------------#####
# MAIN RUN
#####-------------------------------------------------------------#####
# Create a new bucket
minio_credentials = "credentials_python_minio.json"
create_bucket(bucket_name = "dev", minio_credentials = minio_credentials)

# Upload the dummy input file in examples
dummy_folder = "/media/hieunguyen/HNSD01/outdir/TCGA_20240612/idat_20240612"
path_to_dummy_metadata = "/media/hieunguyen/HNSD01/outdir/TCGA_20240612/01_output/sampling_metadata.csv"
dummy_metadata = pd.read_csv(path_to_dummy_metadata, index_col = [0]).drop("X", axis = 1)
all_files = [item for item in pathlib.Path(dummy_folder).glob("*.idat")]

# for input_file in tqdm(all_files[:100]):
# # for input_file in [item for item in all_files if "7fa33bdd-b12b-4f16-8eaf-22b22e1a257f_noid_Grn.idat" in item.name]:
#     file_name = input_file.name
#     base_metadata = {"User": "Hieu Nguyen", 
#                       "Project": "ECD", 
#                       "Sub_project": "TCGA_exploration"}
#     dict_from_metadata = dummy_metadata[dummy_metadata["File.Name"] == file_name].squeeze().to_dict()
#     input_metadata = {**base_metadata, **dict_from_metadata}
    
#     # lower cases all letters in keys
#     input_metadata = {k.lower(): v for k, v in input_metadata.items()}
    
#     upload_file_with_metadata(
#         bucket_name = "dev",
#         object_name = file_name,
#         file_to_upload = input_file,
#         metadata = input_metadata, 
#         # <<<< FIX ME HIEU: should have a smart way to define metadata and restrict metadata template.
#         # maybe if the bucket is new, the first input metadata will be the template for the bucket.
#         # if the bucket already exists, the metadata should be validated against the template.
#         minio_credentials = minio_credentials)

# extract metadata from bucket
metadatadf = extract_metadata_of_all_objects(bucket_name = "dev", minio_credentials = minio_credentials, simplified = True)
b

Bucket 'dev' already exists.
Metadata extracted successfully.


In [6]:
keys

{'url': 'http://localhost:9001/api/v1/service-account-credentials',
 'accessKey': 'oUEuDDES38CCJ4J43Cq8',
 'secretKey': 'ZsLQ1LA3KL1joPXtf6tWzJXkB5kd57RuXbwTqHMs',
 'api': 's3v4',
 'path': 'auto'}

In [20]:
search_criteria = ".idat"
with open(minio_credentials, 'r') as file:
        keys = json.load(file)
    
# Initialize MinIO client
minio_client = minio.Minio(
        "localhost:9000",
        access_key=keys['accessKey'],
        secret_key=keys['secretKey'],
        secure=False
    )
    
# List all buckets
buckets = minio_client.list_buckets()
    
# Initialize a dictionary to hold search results
search_results = {}
    
# Iterate through each bucket
for bucket in buckets[1:3]:
    print("searching in {}".format(bucket))
    bucket_name = bucket.name
    # List or search objects in the bucket
    objects = minio_client.list_objects(bucket_name, prefix=search_criteria, recursive=True)
    # Compile results
    for obj in objects:
        print(obj)
        if search_criteria in obj.object_name:
            if bucket_name not in search_results:
                search_results[bucket_name] = []
                search_results[bucket_name].append(obj.object_name)

searching in dev
searching in emlinh
